In [1]:
import sqlite3

# Create a database connection
con = sqlite3.connect("nmn.db")
con.execute("PRAGMA foreign_keys = ON")
con.executescript(
    """
CREATE TABLE IF NOT EXISTS ComponentCategories (
    id INTEGER PRIMARY KEY,
    name TEXT,
    content BLOB
);
CREATE TABLE IF NOT EXISTS Components (
    id INTEGER PRIMARY KEY,
    category TEXT,
    score_id INTEGER,
    content BLOB,
    FOREIGN KEY (category) REFERENCES ComponentCategories(name),
    FOREIGN KEY (score_id) REFERENCES Scores(id)
);
CREATE TABLE IF NOT EXISTS Scores (
    id INTEGER PRIMARY KEY,
    content BLOB
);
"""
)

In [6]:
from pathlib import Path
import sys

lib_path = str(Path("../lib").resolve())
if lib_path not in sys.path:
    sys.path.insert(0, lib_path)

In [7]:
from component_classification import (
    generate_categories,
    generate_components,
    classify_components,
)

paths, categories = generate_categories()
components = generate_components()
results = classify_components(components, categories)

In [8]:
# add categories to database

categories_table = [
    (path.name, category.tobytes()) for path, category in zip(paths, categories)
]

with con:
    con.executemany(
        "INSERT INTO ComponentCategories (name, content) VALUES (?, ?)",
        categories_table,
    )

In [10]:
components_table = [(paths[res].name,) for component, res in zip(components, results)]

(18216, 18216)

array([ 8, 36, 22, 45, 54, 49, 54, 11, 36,  7])